In [1]:
import numpy as np

In [2]:
def bond_price(nominal, coupon, ytm, maturity, freq=2):
    total_payments = maturity * freq
    
    coupon_payment = (nominal * coupon) / freq
    
    discount_rate = ytm / freq 
    
    present_coupon_value = 0
    for t in range(1, total_payments + 1):
        present_coupon_value += coupon_payment / (1 + discount_rate) ** t
        
    present_nominal_value = nominal / (1 + discount_rate) ** total_payments
    
    bond_price = present_coupon_value + present_nominal_value
    
    return bond_price


In [3]:
def bond_duration(nominal, coupon, ytm, maturity, freq=2, annual=True):
    price = bond_price(nominal, coupon, ytm, maturity, freq=2)
    
    coupon_payment = (nominal * coupon) / freq
    total_periods = freq * maturity
    period_yield = ytm / freq
    
    # print(coupon_payment, total_periods, period_yield)
    
    numerator = 0
    test = 0.5
    for t in range(1, total_periods + 1):
        if t != total_periods:
            numerator += (t * coupon_payment) / (1 + period_yield) ** t
            test += 0.5
        else: 
            numerator += (total_periods * (nominal + coupon_payment)) / (1 + period_yield) ** total_periods

    duration = numerator / price
    duration = duration / freq if not annual else duration
    
    mod_duration = duration / (1 + (ytm / freq))
    
    return duration, mod_duration
    

In [5]:
def bond_convexity(nominal, coupon, ytm, maturity, freq=2, dy=0.01):
    ytm_minus = ytm - dy
    price_minus = bond_price(nominal, coupon, ytm_minus, maturity, freq=2)
    
    ytm_plus = ytm + dy
    price_plus = bond_price(nominal, coupon, ytm_plus, maturity, freq=2)
    
    price = bond_price(nominal, coupon, ytm, maturity, freq=2)

    convexity = (price_plus + price_minus - 2 * price) / (price * dy ** 2)
    
    return convexity


In [6]:
nominal = 100
coupon = 0.06
ytm = 0.08
maturity = 5 

price = bond_price(nominal, coupon, ytm, maturity, freq=2)
duration, mod_dur =  bond_duration(nominal, coupon, ytm, maturity, freq=2, annual=True)
convexity = bond_convexity(nominal, coupon, ytm, maturity, freq=2)

estimation = price * convexity * (0.15  ** 2) / 2 - (price * mod_dur * 0.01)

In [7]:
price, duration, convexity

(91.88910422064495, 8.722915734073045, 21.157201760197207)

In [8]:
estimation

14.16418440912527

$$
\frac{{\text{{Annual Interest Payment}} + \frac{{\text{{Face Value}} - \text{{Current Price}}}}{{\text{{Years to Maturity}}}}}}{{\frac{{\text{{Face Value}} + \text{{Current Price}}}}{2}}}
$$


In [34]:
def bond_ytm(nominal, market_price, coupon, maturity):
    coupon_payment = nominal * coupon
    
    numerator = coupon_payment + ((nominal - market_price) / maturity)
    denominator = (nominal + market_price) / 2
    
    ytm = numerator / denominator
    
    return ytm
    

In [35]:
bond_ytm(100, 94, 0.05, 10)

0.057731958762886594